# Генерація Ексельок "Виплати Додаткової Допомоги" для вийськових формувань ЗСУ

In [1]:
from packaging.version import Version, parse

import pandas as pd

try:
    assert Version(pd.__version__) < Version("2.0.0")
except AssertionError:
    %pip uninstall pandas -y
    %pip install pandas==1.5.0
    import pandas as pd
    

assert Version(pd.__version__) < Version("2.0.0")

In [2]:
try:
    import numpy as np
except ModuleNotFoundError:
    %pip install numpy
    import numpy as np

In [3]:
try:
    import openpyxl as ox
except ModuleNotFoundError:
    %pip install openpyx
    import openpyxl as ox

In [4]:
from itertools import chain

import warnings
warnings.filterwarnings('ignore')

Цей генератор оперує ексель файлом (див `calendar.xlsx` як приклад) і на основі нього має здатність генерувати таблицю для __рапорту на виплату додаткової винагороди__. Ведення такого файлу дає змогу не витрачати часу на підрахунок особового складу, як курчат по переліку БРок в кінці місяця, потребує мінімальних знать з програмування та екселю/гугл таблиць і дає час людям що ведуть канцелярію витрачати час на більш корисні заняття, але потребує вести ексель файл щодня (або принамні заповнити його перед генерацією) та відформатувати згенеровані таблчики під час подачі рапорту. 

#### Додавання нового місяця

1. Скопіюйте існуючий листок з попереднім місяцем 
2. Додайте чи приберіть додаткові дні (див колонки з нумерацією)
3. Очистіть заповнені дані
4. Починайте вводити дані

In [5]:
# не прибирайте цей коментарій - ці змінні потрібни щоб программа могла дізнатись звідки читати дані (нам потрібен сам файл і імя листкаа у ньому)

# імя листка у книзі з даними за поточний місяць
sheet_name = "квітень"  
excel_file = "calendar.xlsx"

In [6]:
# перевірка того як виглядають данні

df = pd.read_excel(excel_file, sheet_name=sheet_name, header=0)
df.head(5)

,vzvod,title,rank,posada,category,med_cat,name,dob,phone,1,...,21,22,23,24,25,26,27,28,29,30
0,0,Рота чогось там (Управління),капітан,Командир роти,офіцер,NaN,БІДЮК Ігор Самуїлович,1995-06-28,NaN,1111,...,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111
1,0,Рота чогось там (Управління),старший лейтенант,Заступник командира роти,офіцер,NaN,НАЗАРЕВИЧ Богдан Владиславович,1981-10-12,NaN,2222,...,2222,2222,2222,2222,2222,2222,2222,2222,2222,2222
2,0,Рота чогось там (Управління),капітан,Заступник командира роти з морально-психологіч...,офіцер,NaN,СУПРУНЮК Тимур Кирилоович,1986-02-23,NaN,3333,...,ППД,ППД,ППД,2222,2222,2222,2222,2222,2222,2222
3,0,Рота чогось там (Управління),сержант,Головний сержант,сержант,NaN,ОВДІЄНКО Лук’ян Харитонович,1989-11-19,NaN,1111,...,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111
4,0,Рота чогось там (Управління),солдат,Сержант з матеріального забезпечення,сержант,NaN,СМАЛЬ Ян Тамерланович,1982-05-29,NaN,3333,...,3333,3333,ППД,ППД,ППД,ППД,ППД,ППД,ППД,ППД


In [7]:
# якщо ви бажаєте додати свої стани або скорочення, зробіть це тут, 
# нам умовно знадобляться лише числові номера бойових розпоряджень.

BRs = {
    1111:   "БР № 1111 від 11.01.2023",
    2222:   "БР № 2222 від 22.02.2023",
    3333:   "БР № 3333 від 01.03.2022",
    
    # умовні скорочення для індикаціх стану особового складу (ви можете їх змінити і використовувати їх для чогось ще, напрклад 
    # генерації бчс або щоденного зведення по підрозділу
    "ШПИТ" : "Шпиталь",
    "РЕАБ" : "Реабілітація",
    "ППД"  : "Постійний Пункт Дислокації",
    "ВІДП" : "Відпустка",
    "КОМ"  : "Відрядження",
    "СЗЧ"  : "Самовільне залишення частини",
      "-"  : "200ті"
}

In [8]:
# Скатегоризуйте номера соїх БРок (що на 100тку що на 30тку)

types = {
    100:('2222', '1111'), 
    30: ('3333'),
}

# починаємо рахувать з 
day_from   = 1

# кількісь днів у місяці (поставте потрібну кількість власноруч)
days_total = 30 

def br_type(i):
    if str(i) in types[100]: return "100"
    if str(i) in types[30]: return "30"
    
    return 'non' 

# форматування дати вручну. 
def date(day): return f"2023-04-{day:=02d}"

In [9]:
ranges = []
for i, row in df.iterrows():
    
    prev_position_loc  = row[1]
    prev_position_date = 1
    
    for day in range(day_from, days_total+1):
        
        if str(row[day]) == str(prev_position_loc):
            continue
            
        ranges.append((row['rank'], row['name'], prev_position_loc, br_type(prev_position_loc), date(prev_position_date), date(day-1)))
        # saves state and switch state
        prev_position_loc = row[day]
        prev_position_date= day
        
    
    # last day to save 
    if str(row[days_total]) == str(prev_position_loc):
        ranges.append((row['rank'], row['name'], prev_position_loc, br_type(prev_position_loc), date(prev_position_date), date(days_total)))

In [10]:
# створюємо датафрейм
df = pd.DataFrame(ranges, columns=[ "rank", "name", "pos", "type", "from", "to" ])
# конвертуємо дати у тип дати
df[['from','to']] = df[['from','to']].apply(pd.to_datetime)
# додаємо колонку днів
df["days_total"] = ((df['to'] - df['from']) / np.timedelta64(1, 'D') + 1).astype(int)
# вписуємо нотаток щодо бойового розпорядження
df["note"] = df["pos"].apply(lambda x: BRs.get(x, "У переліку БР відсутній опис"))

del df["pos"]

In [11]:
from openpyxl.styles import Border, Side, Alignment

alignment_default=Alignment(horizontal='left', vertical='center', text_rotation=0, wrap_text=True, shrink_to_fit=False, indent=1)
alignment_center=Alignment(horizontal='center', text_rotation=0, wrap_text=True, shrink_to_fit=False)


border = Border(left   = Side(border_style=None, color='00000000'),
                right  = Side(border_style=None, color='00000000'),
                top    = Side(border_style=None, color='FF000000'),
                bottom = Side(border_style=None, color='FF000000'))

In [12]:
wb = ox.load_workbook('template.xlsx')


def style(cell, alignment=None):
    cell.alignment = alignment if alignment else alignment_default
    cell.border    = border

    
for t in [100, 30]:
    # df[(df["type"] == t)].to_excel(writer, sheet_name=str(t), index=False)
    sheet = wb.worksheets[wb.sheetnames.index(str(t))]
    data = df[(df["type"] == str(t))]
    
    data.insert(0, "n", "")
    del data["type"]
    
    data['to'] = data['to'].dt.strftime('%Y-%m-%d')
    data['from'] = data['from'].dt.strftime('%Y-%m-%d')
    
    sizeof = data.shape[0]
    
    [sheet.append(row) for row in data.values.tolist()]
    if not sizeof:
        continue
    
    
    seen, counter = ["", 0, 0], 1
    for column in sheet[f"C3:C{sizeof+2}"]:
        key = (column[0].value, sheet[f"b{column[0].row}"].value)
        if key == seen[0]:
            
            [style(cell, alignment=alignment_center) for cell in chain.from_iterable(sheet[f'D{column[0].row}:F{column[0].row}'])]
            
            seen[2] = column[0].row
            continue
        
        if seen[1] != seen[2]:
            sheet.merge_cells(f'C{seen[1]}:C{seen[2]}')
            sheet.merge_cells(f'B{seen[1]}:B{seen[2]}')
            sheet.merge_cells(f'A{seen[1]}:A{seen[2]}')
            [style(cell, alignment=alignment_center) for cell in 
                 chain.from_iterable(sheet[f'D{seen[1]}:F{seen[2]}'])]    
            
        
        # reset index
        seen = [key, column[0].row, column[0].row]
        sheet[f'A{column[0].row}'] = counter
        counter += 1
            
        [style(cell) for cell in chain.from_iterable(sheet[f'A{column[0].row}:C{column[0].row}'])]
        [style(cell, alignment=alignment_center) for cell in chain.from_iterable(sheet[f'D{column[0].row}:F{column[0].row}'])]
    
wb.save(f"винагорода-{sheet_name}.xlsx")